<a href="https://colab.research.google.com/github/Nikhilsai-123/dailyassesment/blob/main/pspyk3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PySpark Practice Notebook
Experimenting further with Spark Dataframes

In [ ]:
#Create CSV files for the following Datasets

patients
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

visits
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

In [10]:
%%writefile patients.csv
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

Writing patients.csv


In [11]:
%%writefile visits.csv
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

Writing visits.csv


In [8]:
#Import necessary spark libraries
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=9bf995f5ff938e54d388a6836a32a4ecdc40df8cdcf97dbaaa6e1f9e66d97919
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [12]:
#Create a SparkSession
spark=SparkSession.builder.appName("SparkPractice").getOrCreate()
df=spark.read.option('header',True).csv('/content/patients.csv')
df1=spark.read.option('header',True).csv('/content/visits.csv')

In [15]:
df.show()
df1.show()
#Create dataframes from CSV files using enforced Schema
patientSchema=StructType([
    StructField('patientId',IntegerType(),True)
    StructField('firstName',IntegerTYpe(),True)
    StructField('lastName',IntegerType(),True)
    StructField('age',IntegerType(),True)
])
visitSchema=StructType([
    StructField('visitId',IntegerType(),True)
    StructField('patientId',IntegerType(),True)
    StructField('visitDuration',IntegerType(),True)
])
#Schems Directive [Names as string, everything else as integers]


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-15-3c9776dd18bc>, line 5)

In [ ]:
# prompt: Create dataframes from CSV files using enforced Schema

patientSchema = StructType([
    StructField("patientId", IntegerType(), True),
    StructField("firstName", StringType(), True),
    StructField("lastName", StringType(), True),
    StructField("age", IntegerType(), True)
])

visitSchema = StructType([
    StructField("visitId", IntegerType(), True),
    StructField("patientId", IntegerType(), True),
    StructField("visitDuration", IntegerType(), True)
])

df_patients = spark.read.option("header", True).schema(patientSchema).csv("/content/patients.csv")
df_visits = spark.read.option("header", True).schema(visitSchema).csv("/content/visits.csv")

df_patients.show()
df_visits.show()


In [ ]:
#Display all the valid visits


#Fetch the total number of valid visits


In [ ]:
#Get the total hours of patient visit


In [ ]:
#Fetch the top two most visited patients on the basis of occurances


In [ ]:
#Add a new column patientFullName & display complete dataframe


In [ ]:
#Display the name of patients with maximum visiting hours


In [ ]:
#Find the patient with maximum visiting hours in a single visit


In [ ]:
#Show the patient info according to age-seniority
